
### Modelos de lenguaje n-gramas

Predecir es difícil, especialmente sobre el futuro. Pero, ¿qué tal predecir algo que parece mucho más fácil, como las próximas palabras que alguien va a decir? ¿Qué palabra, por ejemplo, es probable que siga a:

 **Please turn your homework ...**

Esperemos que la mayoría de ustedes haya concluido que una palabra muy probable es "in", o posiblemente "over", pero probablemente no "refrigerador" o "the". En esta semana formalizamos esta intuición introduciendo modelos que asignan una probabilidad a cada palabra siguiente posible.
Los modelos que asignan probabilidades a las palabras que vienen a continuación, o a secuencias de palabras en general, se llaman modelos de lenguaje o LMs. 

¿Por qué querríamos predecir palabras futuras? 

Resulta que los grandes modelos de lenguaje que revolucionaron el procesamiento moderno del lenguaje natural están entrenados simplemente prediciendo palabras.  Como veremos luego, los grandes modelos de lenguaje aprenden una cantidad enorme sobre el lenguaje únicamente por estar entrenados para predecir las palabras siguientes a partir de palabras vecinas.

Los modelos de lenguaje también pueden asignar una probabilidad a una oración completa. Por ejemplo, pueden predecir que la siguiente secuencia tiene una probabilidad mucho mayor de aparecer en un texto:

**all of a sudden I notice three guys standing on the sidewalk**

que esta misma secuencia de palabras en un orden diferente:

**on guys all I of notice sidewalk three a sudden standing the**

¿Por qué importa cuál es la probabilidad de una oración o cuán probable es la siguiente palabra? 

En muchas aplicaciones de procesamiento del lenguaje natural podemos usar la probabilidad como una forma de elegir una oración o una mejor palabra  sobre una menos apropiada. Por ejemplo, podemos corregir errores gramaticales o de ortografía como "Their are two midterms", en el que "There" se escribió incorrectamente como "Their", o "Everything has improve", en el que "improve" debería haber sido "improved". 

La frase "There are" será mucho más probable que "Their are", y "has improved" que "has improve", lo que permite a un modelo de lenguaje ayudar a los usuarios a seleccionar la variante más gramatical. 

O para que un sistema de reconocimiento de voz se dé cuenta de que dijiste "I will be back soonish" y no "I will be bassoon dish", ayuda saber que "back soonish" es una secuencia mucho más probable. 

Los modelos de lenguaje también pueden ayudar en sistemas de comunicación aumentativa y alternativa. Las personas suelen usar dispositivos AAC si no pueden hablar o hacer señas físicamente, pero pueden en su lugar usar la mirada o otros movimientos específicos para seleccionar palabras de un menú. La predicción de palabras se puede usar para sugerir palabras probables para el menú.


**N-Gramas**

Comencemos con la tarea de calcular $P(w|h)$, la probabilidad de una palabra $w$ dada una historia $h$. Supongamos que la historia $h$ es "its water is so transparent that" y queremos saber la probabilidad de que la siguiente palabra sea "el(the)":

$P(\text{the} \mid \text{its water is so transparent that})$ 

Una forma de estimar esta probabilidad es a partir de los recuentos de frecuencias relativas: tomar un corpus muy grande, contar el número de veces que vemos "its water is so transparent that" y contar el número de veces que esto es seguido por "the". Esto sería responder a la pregunta "De las veces que vimos la historia `h`, ¿cuántas veces fue seguida por la palabra `w`?, como sigue:


$$
P(\text{the}|\text{its water is so transparent that}) = \frac{C(\text{its water is so transparent that the})}{C(\text{its water is so transparent that})}
$$


Con un corpus lo suficientemente grande, como la web, podemos calcular estos recuentos y estimar la probabilidad a partir de la ecuación anterior.
Aunque este método de estimar probabilidades directamente a partir de recuentos funciona bien en muchos casos, resulta que ni siquiera la web es lo suficientemente grande para darnos buenas estimaciones en la mayoría de los casos. Esto se debe a que el lenguaje es creativo; se crean nuevas oraciones todo el tiempo, y no siempre podremos contar oraciones completas. Incluso simples extensiones de la oración de ejemplo pueden tener recuentos de cero en la web.

De manera similar, si quisiéramos conocer la probabilidad conjunta de toda una secuencia de palabras como "its water is so transparent", podríamos hacerlo preguntando "de todas las posibles secuencias de cinco palabras, ¿cuántas de ellas son 'its water is so transparent?". Tendríamos que obtener el recuento de "its water is so transparent" y dividir por la suma de los recuentos de todas las posibles secuencias de cinco palabras. 

Por esta razón, necesitaremos introducir formas más inteligentes de estimar la probabilidad de una palabra `w` dada una historia `h`, o la probabilidad de toda una secuencia de palabras `W`. 

Comencemos formalizando un poco la notación. Para representar la probabilidad de que una variable aleatoria $X_i$ tome el valor "the", o $P(X_i = "the")$, utilizaremos la simplificación $P(\text{the})$. 

Representaremos una secuencia de $n$ palabras ya sea como $w_1 \dots w_n$ o $w_{1:n}$. Así, la expresión $w_{1:n-1}$ significa la cadena $w_1,w_2,\dots,w_{n-1}$, pero también utilizaremos la notación equivalente $w_{<n}$, que puede leerse como "todos los elementos de $w$ desde $w_1$ hasta e incluyendo $w_{n-1}$". 

Para la probabilidad conjunta de que cada palabra en una secuencia tenga un valor particular $P(X_1 = w_1, X_2 = w_2, X_3 = w_3,\dots, X_n = w_n)$ utilizaremos $P(w_1, w_2,\dots, w_n)$.

Ahora, ¿cómo podemos calcular las probabilidades de secuencias completas como $P(w_1, w_2,\dots, w_n)$? 

Una cosa que podemos hacer es descomponer esta probabilidad utilizando la regla de la cadena de probabilidad:

$$P(X_1 \dots X_n) = P(X_1)P(X_2 | X_1) P(X_3 | X_{1:2}) \dots P(X_n | X_{1:n-1}) = \prod_{k=1}^{n} P(X_k | X_{1:k-1})$$

Aplicando la regla de la cadena a las palabras, obtenemos

$$P(w_{1:n}) = P(w_1)P(w_2 | w_1)P(w_3 | w_{1:2}) \dots P(w_n | w_{1:n-1}) \\
= \prod_{k=1}^{n} P(w_k | w_{1:k-1})
$$

La regla de la cadena muestra el vínculo entre el cálculo de la probabilidad conjunta de una secuencia y el cálculo de la probabilidad condicional de una palabra dada las palabras anteriores. 

No conocemos ninguna forma de calcular la probabilidad exacta de una palabra dada una larga secuencia de palabras precedentes, $P(w_n|w_{1:n-1})$. 

El **modelo bigrama**, por ejemplo, aproxima la probabilidad de una palabra dada todas las palabras anteriores $P(w_n|w_{1:n-1})$ utilizando solo la probabilidad condicional de la palabra anterior $P(w_n|w_{n−1})$. En otras palabras, en lugar de calcular la probabilidad:

$P(\text{the}|\text{Walden Pond's water is so transparent that})$

lo aproximamos con la probabilidad:$P(\text{the}|\text{that})$

Cuando usamos un modelo bigrama para predecir la probabilidad condicional de la siguiente palabra, estamos haciendo la siguiente aproximación:

$P(w_n|w_{1:n-1}) \approx P(w_n|w_{n-1})$ 

La suposición de que la probabilidad de una palabra depende solo de la palabra anterior se llama **suposición de Markov**. 
Los modelos de Markov son la clase de modelos probabilísticos que asumen que podemos predecir la probabilidad de alguna unidad futura sin mirar demasiado atrás.


Los modelos bigrama (que mira una palabra en el pasado) se pueden generalizar para considerar palabras más anteriores, como los trigramas (que consideran dos palabras anteriores) y en general, los n-gramas, que miran `n-1` palabras hacia el pasado.

Veamos una ecuación general para esta aproximación de n-gramas a la probabilidad condicional de la siguiente palabra en una secuencia. Usamos `N` para indicar el tamaño del n-grama, por lo que `N = 2` significa bigramas y `N = 3` significa trigramas. 

Entonces, aproximamos la probabilidad de una palabra dado todo su contexto de la siguiente manera:

$P(w_n | w_{1:n-1}) \approx P(w_n | w_{n-N+1:n-1})$

Dada la suposición del bigrama para la probabilidad de una palabra individual, podemos calcular la probabilidad de una secuencia completa de palabras sustituyendo la ecuación anterior en la ecuación general de la probabilidad de una secuencia:

$P(w_{1:n}) \approx \prod_{k=1}^{n} P(w_k | w_{k-1})$

Un método intuitivo es la estimación por máxima verosimilitud (MLE). Obtenemos la estimación MLE para los parámetros de un modelo n-grama obteniendo conteos de un corpus y normalizando esos conteos para que estén entre `0` y `1`.

**Para los modelos probabilísticos, normalizar significa dividir por algún conteo total de manera que las probabilidades resultantes estén entre 0 y 1.**

Por ejemplo, para calcular la probabilidad de un bigrama particular de una palabra $w_n$ dado $w_{n-1}$, calculamos el conteo del bigrama $C(w_{n-1} w_n)$ y lo normalizamos dividiendo entre la suma de todos los bigramas que comparten la misma primera palabra $w_{n-1}$:

$P(w_n | w_{n-1}) = \frac{C(w_{n-1}w_n)}{\sum_{w} C(w_{n-1} w)}$

Podemos simplificar esta ecuación, ya que la suma de todos los conteos de bigramas que comienzan con una palabra $w_{n-1}$ debe ser igual al conteo unigrama para esa palabra $w_{n-1}$:

$P(w_n | w_{n-1}) = \frac{C(w_{n-1} w_n)}{C(w_{n-1})}$


### Ejemplos

Vamos a trabajar con un ejemplo utilizando un mini-corpus de tres oraciones. Primero necesitaremos aumentar cada oración con un símbolo especial `<s>` al principio de la oración, para darnos el contexto de bigrama de la primera palabra. También necesitaremos un símbolo especial de fin de oración `</s>`. 

**Necesitamos el símbolo de fin de oración para que la gramática bigrama sea una verdadera distribución de probabilidad. Sin un símbolo de fin de oración, en lugar de que las probabilidades de todas las oraciones sumen uno, las probabilidades de las oraciones de una longitud dada sumarían uno. Este modelo definiría un conjunto infinito de distribuciones de probabilidad, con una distribución por cada longitud de oración.**

`<s> I am Sam</s>`

`<s> Sam I am</s>`

`<s> I do not like green eggs and ham </s>`

Aquí están los cálculos para algunas de las probabilidades de bigramas a partir de este corpus:


$P(\text{I}|\texttt{<s>}) = \frac{2}{3} = 0.67$

$P(\text{Sam}|\texttt{<s>}) = \frac{1}{3} = 0.33$

$P(\text{am}|\text{I}) = \frac{2}{3} = 0.67$

$P(\texttt{</s>}|\text{Sam}) = \frac{1}{2} = 0.5$

$P(\text{Sam}|\text{am}) = \frac{1}{2} = 0.5$

$P(\text{do}|\text{I}) = \frac{1}{3} = 0.33$


Para el caso general de la estimación de parámetros de MLE n-gramas se tiene:

$P(w_n|w_{n-N+1:n-1}) = \frac{C(w_{n-N+1:n-1} w_n)}{C(w_{n-N+1:n-1)}}$

Esta ecuacion estima la probabilidad de n-grama dividiendo la frecuencia observada de una secuencia particular por la frecuencia observada de un contexto anterior (prefijo). 

Esta relación se llama **frecuencia relativa**. 
    
La idea detrás de esta ecuación es que la probabilidad de que ocurra una palabra en un contexto específico se puede aproximar observando con qué frecuencia aparece ese contexto completo (incluyendo la palabra) dividido por la frecuencia del prefijo (el contexto sin la última palabra). Esto refleja cómo de probable es que una palabra específica siga un conjunto particular de palabras anteriores (el prefijo).

Dijimos anteriormente que el uso de frecuencias relativas como una forma de estimar probabilidades es un ejemplo de estimación de máxima verosimilitud o `MLE`. 
En `MLE`, el conjunto resultante de parámetros maximiza la verosimilitud del conjunto de entrenamiento `T` dado el modelo `M` (es decir, $P(T |M)$). 
    

### Ejemplo

1. El ejemplo se enfoca en desarrollar un sistema de procesamiento de lenguaje natural que pueda calcular probabilidades de palabras usando modelos de unigramas, bigramas y trigramas a partir del corpus del Berkeley Restaurant Project.

Se usará la técnica de `MLE` y se aplicarán técnicas de suavizado como Laplace para manejar problemas de palabras fuera de vocabulario (OOV).

2. Pasos a seguir

Carga del corpus

Utilizar el corpus del Berkeley Restaurant Project, que contiene datos de diálogos sobre restaurantes. Se debe procesar el corpus para tokenizar el texto y dividirlo en palabras.

    * Tokenización: Convertir el texto en una lista de palabras.
    * Normalización: Pasar todo a minúsculas y remover signos de puntuación.

In [ ]:
import re
from collections import defaultdict

def tokenize(text):
    # Tokeniza el texto y remueve signos de puntuación
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text.split()

# Ejemplo de tokenización
corpus = """
    What restaurants serve Italian food in Berkeley?
    Where can I find a cheap restaurant downtown?
"""
tokens = tokenize(corpus)
print(tokens)


2.2. Construcción de tablas de frecuencia

Crear tablas de frecuencia para unigramas, bigramas y trigramas. Los conteos de estas secuencias son fundamentales para construir las probabilidades condicionales.


In [ ]:
from collections import Counter

def build_ngram_counts(tokens, n):
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return Counter(ngrams)

# Conteo de unigramas, bigramas y trigramas
unigrams = build_ngram_counts(tokens, 1)
bigrams = build_ngram_counts(tokens, 2)
trigrams = build_ngram_counts(tokens, 3)

print("Unigramas:", unigrams)
print("Bigramas:", bigrams)
print("Trigramas:", trigrams)


Línea clave: `ngrams = zip(*[tokens[i:] for i in range(n)])`

1. `tokens[i:] for i in range(n)`: Crea una lista de sublistas (o iteradores) que son versiones desplazadas de la lista original de tokens. Por ejemplo, si tenemos los tokens `['a', 'b', 'c', 'd']` y `n=2` (bigramas), esto generará dos versiones:

- `tokens[0:] --> ['a', 'b', 'c', 'd']` (sin desplazamiento)
- `tokens[1:] → ['b', 'c', 'd']` (desplazado en una posición)

2. `zip(*...)`: Combina las sublistas generadas en el paso anterior, de manera que produce tuplas que representan las secuencias de n-gramas. Siguiendo el ejemplo de bigramas, zip tomaría un elemento de cada lista desplazada y los combinaría en tuplas:

- La primera tupla sería `('a', 'b')`
- La segunda tupla sería `('b', 'c')`
- La tercera tupla sería `('c', 'd')`

Para trigramas, el proceso es similar pero con tres listas desplazadas.

3. `Counter(ngrams)`: Después de crear las tuplas de n-gramas con zip, se las pasa a Counter, que cuenta cuántas veces aparece cada tupla (n-grama) en el texto.

2.3. MLE (Maximum Likelihood Estimation)

Usar las ecuaciones de MLE para calcular la probabilidad condicional de una palabra dado el contexto.

**Probabilidad de un bigrama**

$P(w_n | w_{n-1}) = \frac{C(w_{n-1} w_n)}{C(w_{n-1})}$

In [ ]:
def bigram_prob(bigrams, unigrams, word1, word2):
    return bigrams[(word1, word2)] / unigrams[(word1,)]

# Ejemplo de probabilidad de bigrama
word1, word2 = 'cheap', 'restaurant'
print(f"P({word2}|{word1}) =", bigram_prob(bigrams, unigrams, word1, word2))

2.4. Aplicación de la fórmula general de n-gramas

Generalización para cualquier n-grama usando la fórmula:

$P(w_n|w_{n-N+1:n-1}) = \frac{C(w_{n-N+1:n-1} w_n)}{C(w_{n-N+1:n-1)}}$


In [ ]:
def ngram_prob(ngram_counts, lower_order_counts, context, word):
    return ngram_counts[context + (word,)] / lower_order_counts[context]

# Ejemplo con trigramas
context = ('i', 'find')
word = 'a'
print(f"P({word}|{' '.join(context)}) =", ngram_prob(trigrams, bigrams, context, word))


2.5. Suavizado de Laplace

Para manejar el problema de OOV y asegurar que ninguna probabilidad sea cero, se aplicará suavizado de Laplace:

$P(w_n | w_{n-1}) = \frac{C(w_{n-1} w_n) + 1}{C(w_{n-1}) + |V|}$

In [ ]:
from collections import defaultdict

# Función para aplicar suavizado de Laplace
def laplace_smoothing(bigrams, unigrams, word1, word2, vocab_size):
    """
    Calcula la probabilidad suavizada de un bigrama usando el suavizado de Laplace.
    
    
    Parámetros:
    - bigrams: un diccionario con conteos de bigramas.
    - unigrams: un diccionario con conteos de unigramas.
    - word1: la primera palabra del bigrama.
    - word2: la segunda palabra del bigrama.
    - vocab_size: el tamaño del vocabulario (número total de palabras únicas).
    
    Retorna:
    - La probabilidad suavizada del bigrama.
    """
    bigram_count = bigrams[(word1, word2)]
    unigram_count = unigrams[(word1,)]
    return (bigram_count + 1) / (unigram_count + vocab_size)

# Ejemplo de uso
unigrams = defaultdict(int)
bigrams = defaultdict(int)

# Ejemplo de corpus tokenizado
tokens = ['i', 'find', 'a', 'cheap', 'restaurant', 'downtown']

# Construcción de conteos de unigramas y bigramas
for i in range(len(tokens) - 1):
    unigrams[(tokens[i],)] += 1
    bigrams[(tokens[i], tokens[i+1])] += 1
# No olvides contar el último unigrama
unigrams[(tokens[-1],)] += 1

# Tamaño del vocabulario
vocab_size = len(set(tokens))

# Calculamos la probabilidad suavizada para el bigrama ('cheap', 'restaurant')
word1, word2 = 'cheap', 'restaurant'
prob = laplace_smoothing(bigrams, unigrams, word1, word2, vocab_size)
print(f"Probabilidad suavizada de ('{word1}', '{word2}'):", prob)


2.6. Evaluación del modelo

Para evaluar la calidad del modelo, implementamos la perplejidad como métrica para medir el desempeño del modelo de n-gramas en un conjunto de validación.

In [ ]:
import math

def calculate_perplexity(test_data, bigrams, unigrams):
    perplexity = 0
    N = len(test_data)
    for i in range(1, N):
        word1, word2 = test_data[i-1], test_data[i]
        prob = laplace_smoothing(bigrams, unigrams, word1, word2, vocab_size)
        perplexity += -math.log(prob)
    return math.exp(perplexity / N)

test_tokens = tokenize("find a restaurant downtown")
print("Perplexity:", calculate_perplexity(test_tokens, bigrams, unigrams))


2.7 Interpolación de n-gramas

La interpolación busca combinar modelos de unigramas, bigramas y trigramas para obtener una mejor estimación de las probabilidades. Para cualquier n-grama, la probabilidad interpolada se puede calcular con la fórmula:

$$
P_{interp}(w_n | w_{n-1}, w_{n-2}) = \lambda_3 P(w_n | w_{n-1}, w_{n-2}) + \lambda_2 P(w_n | w_{n-1}) + \lambda_1 P(w_n)
$$

Donde $\lambda_1$, $\lambda_2$, $\lambda_3$ son los pesos asignados a los unigramas, bigramas y trigramas, respectivamente.

In [ ]:
from collections import defaultdict

# Función para obtener la probabilidad interpolada
def interpolated_prob(trigrams, bigrams, unigrams, word1, word2, word3, vocab_size, lambdas=(0.1, 0.3, 0.6)):
    """
    Calcula la probabilidad interpolada usando unigramas, bigramas y trigramas.
    P(w_n | w_{n-2}, w_{n-1}) = λ3 * P(w_n | w_{n-2}, w_{n-1}) + λ2 * P(w_n | w_{n-1}) + λ1 * P(w_n)

    Parámetros:
    - trigrams: diccionario con conteos de trigramas.
    - bigrams: diccionario con conteos de bigramas.
    - unigrams: diccionario con conteos de unigramas.
    - word1: palabra n-2.
    - word2: palabra n-1.
    - word3: palabra n.
    - vocab_size: tamaño del vocabulario.
    - lambdas: pesos para la interpolación.

    Retorna:
    - La probabilidad interpolada.
    """
    # Descomponemos los pesos lambda
    lambda1, lambda2, lambda3 = lambdas

    # Calcular probabilidades individuales (con suavizado Laplace)
    P_w3 = (unigrams[(word3,)] + 1) / (sum(unigrams.values()) + vocab_size)
    P_w3_given_w2 = (bigrams[(word2, word3)] + 1) / (unigrams[(word2,)] + vocab_size)
    P_w3_given_w1_w2 = (trigrams[(word1, word2, word3)] + 1) / (bigrams[(word1, word2)] + vocab_size)

    # Probabilidad interpolada
    return lambda3 * P_w3_given_w1_w2 + lambda2 * P_w3_given_w2 + lambda1 * P_w3

# Ejemplo de datos y uso
tokens = ['i', 'find', 'a', 'cheap', 'restaurant', 'downtown']

# Construimos unigramas, bigramas y trigramas
unigrams = defaultdict(int)
bigrams = defaultdict(int)
trigrams = defaultdict(int)

for i in range(len(tokens) - 2):
    unigrams[(tokens[i],)] += 1
    bigrams[(tokens[i], tokens[i+1])] += 1
    trigrams[(tokens[i], tokens[i+1], tokens[i+2])] += 1
unigrams[(tokens[-2],)] += 1
bigrams[(tokens[-2], tokens[-1])] += 1
unigrams[(tokens[-1],)] += 1

vocab_size = len(set(tokens))

# Calcular probabilidad interpolada para ("cheap", "restaurant", "downtown")
word1, word2, word3 = 'cheap', 'restaurant', 'downtown'
prob_interpolada = interpolated_prob(trigrams, bigrams, unigrams, word1, word2, word3, vocab_size)
print(f"Probabilidad interpolada de ('{word1}', '{word2}', '{word3}'):", prob_interpolada)


2.8 El suavizado de **Kneser-Ney** 

Es una técnica más avanzada que no solo toma en cuenta los conteos de los n-gramas, sino también la diversidad de contextos en los que aparece una palabra. Este suavizado es particularmente útil cuando ciertos n-gramas son raros pero las palabras en ellos no lo son.

Para un bigrama, la probabilidad suavizada de **Kneser-Ney** es:

$
P_{KN}(w_n | w_{n-1}) = \max(C(w_{n-1} w_n) - d, 0) / C(w_{n-1}) + \lambda(w_{n-1}) P_{KN}(w_n)
$

Donde $d$ es un descuento y $\lambda(w_{n-1})$ ajusta la probabilidad restante para asegurar que las probabilidades sumen `1`.


In [ ]:
def kneser_ney_prob(bigrams, unigrams, word1, word2, discount, vocab_size):
    """
    Calcula la probabilidad suavizada de Kneser-Ney para bigramas.
    
    Parámetros:
    - bigrams: diccionario con conteos de bigramas.
    - unigrams: diccionario con conteos de unigramas.
    - word1: primera palabra del bigrama.
    - word2: segunda palabra del bigrama.
    - discount: el factor de descuento.
    - vocab_size: tamaño del vocabulario.
    
    Retorna:
    - La probabilidad de Kneser-Ney para el bigrama.
    """
    # Contar bigramas y unigramas
    bigram_count = bigrams[(word1, word2)]
    unigram_count = unigrams[(word1,)]

    # Descuento aplicado
    max_term = max(bigram_count - discount, 0) / unigram_count

    # Conteo de bigramas distintos que empiezan con word1 (contexto)
    unique_continuations = len([w for w in unigrams if (word1, w[0]) in bigrams])

    # Probabilidad marginal de la palabra individual (unigrama)
    marginal_prob = (unigrams[(word2,)] + 1) / (sum(unigrams.values()) + vocab_size)

    # Probabilidad suavizada
    return max_term + discount * unique_continuations * marginal_prob / unigram_count

# Parámetros
discount = 0.75  # Un descuento típico en Kneser-Ney

# Calcular la probabilidad suavizada de Kneser-Ney para ('cheap', 'restaurant')
word1, word2 = 'cheap', 'restaurant'
prob_kneser_ney = kneser_ney_prob(bigrams, unigrams, word1, word2, discount, vocab_size)
print(f"Probabilidad de Kneser-Ney para ('{word1}', '{word2}'):", prob_kneser_ney)


### Ejercicios

**Ejercicio 1: Carga y preprocesamiento del corpus**

**Objetivo:** Cargar y preprocesar un corpus de texto.

- **Instrucciones:**
  1. Carga un archivo de texto que contenga diálogos sobre restaurantes (puede simular el corpus del **Berkeley Restaurant Project**).
  2. Implementa una función que procese el texto eliminando signos de puntuación y convierta todo a minúsculas.
  3. Tokeniza el texto en una lista de palabras.

```python
# Archivo corpus.txt
"""
What restaurants serve Italian food in Berkeley?
Where can I find a cheap restaurant downtown?
"""

def load_and_preprocess_corpus(file_path):
    """
    Carga y preprocesa el corpus desde un archivo de texto.
    - Elimina signos de puntuación y convierte todo a minúsculas.
    - Devuelve una lista de palabras (tokens).
    """
    # Implementar la función de carga y preprocesamiento del corpus
    pass

# Tarea: Implementa y prueba la función usando un archivo de texto.
```

**Ejercicio 2: Cálculo decConteos de unigramas, bigramas y trigramas**

**Objetivo:** Implementar un código para calcular los conteos de unigramas, bigramas y trigramas a partir del corpus tokenizado.

- **Instrucciones:**
  1. Escribe una función para contar los unigramas, bigramas y trigramas.
  2. Devuelve los conteos como diccionarios.

```python
from collections import defaultdict

def calculate_ngram_counts(tokens, n):
    """
    Calcula los conteos de n-gramas a partir de una lista de tokens.
    Retorna un diccionario con los conteos de n-gramas.
    """
    # Implementar el cálculo de n-gramas
    pass

# Tarea: Implementa la función para unigramas, bigramas y trigramas
# Ejemplo: tokens = ['what', 'restaurants', 'serve', 'italian', 'food']
```

**Ejercicio 3: Implementación de MLE (Maximum Likelihood Estimation)**

**Objetivo:** Implementar la estimación de máxima verosimilitud (MLE) para bigramas y trigramas.

- **Instrucciones:**
  1. Implementa una función que calcule la probabilidad de un bigrama o trigram usando MLE.
  2. Usa la fórmula:

  $$
  P(w_n | w_{n-1}) = \frac{C(w_{n-1} w_n)}{C(w_{n-1})}
  $$

```python
def mle_prob(bigrams, unigrams, word1, word2):
    """
    Calcula la probabilidad MLE para un bigrama.
    """
    # Implementar la probabilidad MLE
    pass

# Tarea: Calcula P('italian' | 'serve') usando los bigramas y unigramas de tu corpus.
```

**Ejercicio 4: Aplicación de suavizado de Laplace**

**Objetivo:** Implementar el suavizado de Laplace para bigramas.

- **Instrucciones:**
  1. Modifica la función MLE para incluir el suavizado de Laplace, usando la fórmula:

  $$
  P(w_n | w_{n-1}) = \frac{C(w_{n-1} w_n) + 1}{C(w_{n-1}) + |V|}
  $$

  2. Implementa la función y calcula la probabilidad suavizada para algunos bigramas del corpus.

```python
def laplace_smoothing(bigrams, unigrams, word1, word2, vocab_size):
    """
    Aplica el suavizado de Laplace a un bigrama.
    """
    # Implementar suavizado de Laplace
    pass

# Tarea: Calcula la probabilidad suavizada para varios bigramas como ('cheap', 'restaurant').
```

**Ejercicio 5: Implementación de la interpolación de N-gramas**

**Objetivo:** Implementar la interpolación de unigramas, bigramas y trigramas para mejorar las probabilidades.

- **Instrucciones:**
  1. Usa la fórmula de interpolación:

  $$
  P_{interp}(w_n | w_{n-1}, w_{n-2}) = \lambda_3 P(w_n | w_{n-1}, w_{n-2}) + \lambda_2 P(w_n | w_{n-1}) + \lambda_1 P(w_n)
  $$

  2. Implementa una función que combine unigramas, bigramas y trigramas con pesos $\lambda_1$, $\lambda_2$, $\lambda_3$.

```python
def interpolated_prob(trigrams, bigrams, unigrams, word1, word2, word3, vocab_size, lambdas=(0.1, 0.3, 0.6)):
    """
    Calcula la probabilidad interpolada usando unigramas, bigramas y trigramas.
    """
    # Implementar interpolación
    pass

# Tarea: Prueba la interpolación para secuencias de tres palabras del corpus.
```

**Ejercicio 6: Suavizado de Kneser-Ney**

**Objetivo:** Implementar el suavizado de Kneser-Ney para bigramas.

- **Instrucciones:**
  1. Implementa el suavizado de Kneser-Ney para calcular la probabilidad de un bigrama.
  2. Usa la fórmula:

  $$
  P_{KN}(w_n | w_{n-1}) = \frac{\max(C(w_{n-1} w_n) - d, 0)}{C(w_{n-1})} + \lambda(w_{n-1}) P_{KN}(w_n)
  $$

```python
def kneser_ney_prob(bigrams, unigrams, word1, word2, discount, vocab_size):
    """
    Calcula la probabilidad suavizada de Kneser-Ney para bigramas.
    """
    # Implementar suavizado de Kneser-Ney
    pass

# Tarea: Aplica suavizado de Kneser-Ney para calcular la probabilidad de ('cheap', 'restaurant').
```

**Ejercicio 7: Evaluación del modelo con perplejidad**

**Objetivo:** Implementar una función para evaluar el modelo de n-gramas con **perplejidad**.

- **Instrucciones:**
  1. Implementa una función que calcule la **perplejidad** de un modelo n-grama dado un conjunto de prueba.

  $$
  \text{Perplejidad} = \exp \left( - \frac{1}{N} \sum_{i=1}^N \log P(w_i | w_{i-n+1:i-1}) \right)
  $$

```python
import math

def calculate_perplexity(test_data, ngram_model, unigrams, bigrams, trigrams, vocab_size):
    """
    Calcula la perplejidad de un modelo n-grama dado un conjunto de prueba.
    """
    # Implementar la función de perplejidad
    pass

# Tarea: Calcula la perplejidad del modelo con suavizado de Laplace en un conjunto de prueba.
```

In [ ]:
### Tus respuestas

#### Algunos problemas prácticos

Aunque con fines pedagógicos solo hemos descrito modelos de bigramas, en la práctica podríamos usar modelos de trigramas, que se condicionan en las dos palabras anteriores, o modelos de 4-gramas o incluso 5-gramas, cuando hay suficientes datos de entrenamiento. Nótese que para estos n-gramas más grandes, necesitaremos asumir contextos adicionales a la izquierda y derecha del final de la oración. 

Por ejemplo, para calcular probabilidades de trigramas al comienzo de la oración, usamos dos pseudo-palabras para el primer trigrama (es decir, `P(I|<s><s>)`).


Para calcular probabilidades de trigramas al comienzo de una oración, usamos dos pseudo-palabras `<s>` al principio para representar el comienzo de la oración. Esto ayuda a manejar los casos en los que los primeros trigramas no tienen suficientes palabras previas.

#### Ejemplo:
Supongamos que queremos calcular la probabilidad del trigrama "I am happy" al comienzo de una oración. Queremos encontrar `P(happy | I am)`, que es la probabilidad de que "happy" siga a "I am".

**Paso 1: Preparar la oración con pseudo-palabras**

Dado que "I" es la primera palabra de la oración, necesitamos incluir dos pseudo-palabras `<s>` al principio para calcular el primer trigrama. Así, la oración se modifica a:
```
<s> <s> I am happy
```

**Paso 2: Calcular la probabilidad del primer trigrama**
Para el primer trigrama, que es `(<s>, <s>, I)`, calculamos la probabilidad:
```
P(I | <s><s>)
```
Esto representa la probabilidad de que "I" sea la primera palabra después de dos comienzos de oración.

**Paso 3: Calcular probabilidades sucesivas**

El siguiente trigrama es `(<s>, I, am)`, y su probabilidad se calcula como:
```
P(am | <s>I)
```
Finalmente, para el trigrama "I am happy", calculamos:
```
P(happy | I am)
```

**Observación:**

Siempre representamos y calculamos probabilidades de modelos de lenguaje en formato logarítmico como probabilidades logarítmicas. Dado que las probabilidades son (por definición) menores o iguales a 1, mientras más probabilidades multipliquemos juntas, más pequeño se vuelve el producto. Multiplicar suficientes n-gramas juntos resultaría en un desbordamiento numérico. Al usar probabilidades logarítmicas en lugar de probabilidades crudas, obtenemos números que no son tan pequeños.

La suma en el espacio logarítmico es equivalente a multiplicar en el espacio lineal, por lo que combinamos probabilidades logarítmicas sumándolas. El resultado de hacer todo el cálculo y almacenamiento en espacio logarítmico es que solo necesitamos convertir de nuevo a probabilidades si necesitamos reportarlas al final; entonces simplemente tomamos el $\exp$ del logaritmo:

$$p_1 \times p_2 \times p_3 \times p_4 = \exp(\log p_1 + \log p_2 + \log p_3 + \log p_4)$$

En la práctica, usaremos $\log$ para referirnos al logaritmo natural $(ln)$ cuando la base no se especifique.


#### Evaluación de modelos de lenguaje: conjuntos de entrenamiento y prueba

La evaluación del rendimiento de un modelo de lenguaje puede hacerse de dos maneras: la evaluación extrínseca, que mide cómo el modelo mejora una aplicación en la tarea final, y la evaluación intrínseca, que mide la calidad del modelo sin depender de una aplicación, como con la métrica de perplejidad. 

Para evaluar modelos, se usan tres conjuntos de datos: el conjunto de entrenamiento, para aprender los parámetros del modelo; el conjunto de prueba, que evalúa la capacidad de generalización del modelo en datos nuevos y el conjunto de desarrollo, que permite ajustar el modelo sin sesgar los resultados del conjunto de prueba. 

Es crucial evitar que el conjunto de prueba se use en el entrenamiento para obtener resultados precisos y evitar sobreajuste.


#### Evaluación de modelos de lenguaje: Perplejidad

En la práctica, no usamos la probabilidad como métrica para evaluar modelos de lenguaje, sino una función de la probabilidad llamada perplejidad. La perplejidad es una de las métricas más importantes en el procesamiento del lenguaje natural y también la usamos para evaluar modelos de lenguaje neuronales.

La perplejidad (a veces abreviada como PP o PPL) de un modelo de lenguaje en un conjunto de prueba es la probabilidad inversa del conjunto de prueba (una sobre la probabilidad del conjunto de prueba), normalizada por el número de palabras. Por esta razón, a veces se le llama la perplejidad por palabra.

Para un conjunto de prueba $W = w_1 w_2... w_N$:

\begin{equation}
\text{perplexity}(W) = P(w_1 w_2 \dots w_N)^{-\frac{1}{N}}
\end{equation}

\begin{equation}
= \sqrt[N]{\frac{1}{P(w_1 w_2 \dots w_N)}}
\end{equation}

O podemos usar la regla de la cadena para expandir la probabilidad de $W$:

\begin{equation}
\text{perplexity}(W) = \sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i \mid w_1 \dots w_{i-1})}}
\end{equation}


Note que debido a la inversa en la ecuación anterior, cuanto mayor sea la probabilidad de la secuencia de palabras, menor será la perplejidad. Así, cuanto menor sea la perplejidad de un modelo sobre los datos, mejor es el modelo y minimizar la perplejidad es equivalente a maximizar la probabilidad del conjunto de prueba según el modelo de lenguaje. 

¿Por qué la perplejidad usa la probabilidad inversa? Resulta que la inversa surge de la definición original de perplejidad a partir de la tasa de entropía cruzada en la teoría de la información. 

Los detalles del cálculo de la perplejidad de un conjunto de prueba $W$ dependen de qué modelo de lenguaje usemos. Aquí está la perplejidad de $W$ con un modelo de lenguaje de unigramas (solo la media geométrica de las probabilidades de unigramas):

\begin{equation}
\text{perplexity}(W) = \sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i)}}
\end{equation}

La perplejidad de $W$ calculada con un modelo de lenguaje de bigramas sigue siendo una media geométrica, pero ahora de las probabilidades de bigramas:

\begin{equation}
\text{perplexity}(W) = \sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i \mid w_{i-1})}} 
\end{equation}

Lo que generalmente usamos para la secuencia de palabras en las ecuaciones de bigramas o de $W$ es la secuencia completa de palabras en algún conjunto de prueba. Dado que esta secuencia cruzará muchos límites de oraciones, si nuestro vocabulario incluye un token entre oraciones `<EOS>` o marcadores de inicio y fin de oración separados `<s>` y `</s>`, entonces podemos incluirlos en el cálculo de la probabilidad. Si lo hacemos, entonces también incluimos un token por oración en el conteo total de tokens de palabras $N$.

Por ejemplo, si usamos tanto los tokens de inicio como de fin, incluiríamos el marcador de fin de oración `</s>` pero no el marcador de inicio de oración `<s>` en nuestro conteo de $N$. Esto se debe a que el token de fin de oración va seguido directamente por el token de inicio de oración con una probabilidad casi de $1$, por lo que no queremos que la probabilidad de esa transición ficticia influya en nuestra perplejidad.

Es decir esto tiene que ver con cómo tratamos los límites entre las oraciones cuando calculamos la perplejidad de un modelo de lenguaje, en particular con el uso de tokens especiales como marcadores de inicio y fin de oración.

Mencionamos anteriormente que la perplejidad es una función tanto del texto como del modelo de lenguaje: dado un texto $W$, diferentes modelos de lenguaje tendrán diferentes perplejidades. Debido a esto, la perplejidad puede usarse para comparar diferentes modelos de n-gramas. 

Ten en cuenta que al calcular las perplejidades, el modelo de n-gramas, $P$ debe construirse sin ningún conocimiento del conjunto de prueba o cualquier conocimiento previo del vocabulario del conjunto de prueba. Cualquier tipo de conocimiento del conjunto de prueba puede hacer que la perplejidad sea artificialmente baja. 

La perplejidad de dos modelos de lenguaje solo es comparable si utilizan vocabularios idénticos. Una mejora (intrínseca) en la perplejidad no garantiza una mejora (extrínseca) en el rendimiento de una tarea de procesamiento del lenguaje como el reconocimiento de voz o la traducción automática.

No obstante, debido a que la perplejidad generalmente se correlaciona con las mejoras en la tarea, se usa comúnmente como una métrica de evaluación conveniente. Aún así, cuando sea posible, la mejora de un modelo en la perplejidad debería confirmarse mediante una evaluación de extremo a extremo en una tarea real.


In [ ]:
import math
import nltk
from nltk.util import ngrams
from collections import Counter, defaultdict
nltk.download('punkt')

# Simulamos un conjunto de datos de ejemplo con 38 millones de palabras (simplificado aquí)
text = """<s> This is the first sentence . </s> <s> This is the second sentence . </s> <s> This is the third sentence . </s>"""

# Tokenizamos el texto en palabras y mantenemos <s> y </s> como tokens completos
tokens = [token for token in nltk.word_tokenize(text) if token not in ['<', '>']]
tokens = ['<s>' if token == 's' else token for token in tokens]
tokens = ['</s>' if token == '/s' else token for token in tokens]

# Función para calcular la perplejidad del modelo de unigrama
def unigram_perplexity(test_tokens, unigram_probs):
    N = len(test_tokens)
    perplexity = 1
    for token in test_tokens:
        prob = unigram_probs.get(token, 1e-10)  # Evitar probabilidad cero
        perplexity *= 1/prob
    return perplexity ** (1/N)

# Función para calcular la perplejidad del modelo de bigrama
def bigram_perplexity(test_tokens, bigram_probs):
    N = len(test_tokens) - 1  # número de bigramas
    perplexity = 1
    bigrams = list(ngrams(test_tokens, 2))
    for bigram in bigrams:
        prob = bigram_probs.get(bigram, 1e-10)  # Evitar probabilidad cero
        perplexity *= 1/prob
    return perplexity ** (1/N)

# Función para calcular la perplejidad del modelo de trigramas
def trigram_perplexity(test_tokens, trigram_probs):
    N = len(test_tokens) - 2  # número de trigramas
    perplexity = 1
    trigrams = list(ngrams(test_tokens, 3))
    for trigram in trigrams:
        prob = trigram_probs.get(trigram, 1e-10)  # Evitar probabilidad cero
        perplexity *= 1/prob
    return perplexity ** (1/N)

# Creamos un modelo de unigramas, bigramas y trigramas basado en frecuencias
unigrams = Counter(tokens)
bigrams = Counter(ngrams(tokens, 2))
trigrams = Counter(ngrams(tokens, 3))

# Calculamos probabilidades de unigramas
total_unigrams = sum(unigrams.values())
unigram_probs = {word: count/total_unigrams for word, count in unigrams.items()}

# Calculamos probabilidades de bigramas
bigram_probs = {}
for bigram, count in bigrams.items():
    first_word = bigram[0]
    bigram_probs[bigram] = count / unigrams[first_word]

# Calculamos probabilidades de trigramas
trigram_probs = {}
for trigram, count in trigrams.items():
    first_two_words = (trigram[0], trigram[1])
    trigram_probs[trigram] = count / bigrams[first_two_words]

# Simulamos un conjunto de prueba de 1.5 millones de palabras (simplificado)
test_text = """<s> This is the first test sentence . </s> <s> This is another sentence . </s>"""
test_tokens = [token for token in nltk.word_tokenize(test_text) if token not in ['<', '>']]
test_tokens = ['<s>' if token == 's' else token for token in test_tokens]
test_tokens = ['</s>' if token == '/s' else token for token in test_tokens]

# Calculamos la perplejidad para unigramas, bigramas y trigramas
unigram_ppl = unigram_perplexity(test_tokens, unigram_probs)
bigram_ppl = bigram_perplexity(test_tokens, bigram_probs)
trigram_ppl = trigram_perplexity(test_tokens, trigram_probs)

print(f"Perplejidad del modelo de Unigramas: {unigram_ppl}")
print(f"Perplejidad del modelo de Bigramas: {bigram_ppl}")
print(f"Perplejidad del modelo de Trigramas: {trigram_ppl}")


### Perplejidad como factor de ramificación promedio ponderado 

Resulta que la perplejidad también puede considerarse como el factor de ramificación promedio ponderado de un lenguaje. El factor de ramificación de un lenguaje es el número de posibles palabras siguientes que pueden seguir a una palabra. Si tenemos un lenguaje artificial determinista de números enteros cuyo vocabulario consiste en los 10 dígitos `(cero, uno, dos,..., nueve)`, en el cual cualquier dígito puede seguir a cualquier otro dígito, entonces el factor de ramificación de ese lenguaje es 10.

Primero, convencámonos de que si calculamos la perplejidad de este lenguaje artificial de dígitos obtenemos efectivamente $10$. Supongamos que (en el entrenamiento y en la prueba) cada uno de los $10$ dígitos ocurre con exactamente la misma probabilidad $P = 1/10$. Ahora imagina una prueba de los $10$ dígitos que ocurre con exactamente la misma probabilidad.

Escribimos la perplejidad de este caso:

\begin{equation}
\text{perplexity}(W) = P(w_1 w_2 \dots w_N)^{-\frac{1}{N}}
\end{equation}

\begin{equation}
= \left( \frac{1}{10^N} \right)^{-\frac{1}{N}}
\end{equation}

\begin{equation}
= \frac{1}{10^{-1}}
\end{equation}

\begin{equation}
= 10
\end{equation}


Pero supongamos que el número cero es realmente frecuente y ocurre mucho más a menudo que otros números. Supongamos que 0 ocurre 91 veces en el conjunto de entrenamiento, y cada uno de los otros dígitos ocurrió 1 vez cada uno. Ahora vemos el siguiente conjunto de prueba: `0 0 0 0 0 3 0 0 0 0`. Deberíamos esperar que la perplejidad de este conjunto de prueba sea menor, ya que la mayoría de las veces el siguiente número será cero, lo cual es muy predecible, es decir, tiene una alta probabilidad. 

Así, aunque el factor de ramificación sigue siendo $10$, la perplejidad o el factor de ramificación ponderado es menor.


In [ ]:
import math

# Vocabulario de 10 dígitos (0-9)
vocab_size = 10

# Probabilidad uniforme de cada dígito
prob_digit = 1 / vocab_size

# Simulamos una secuencia de prueba
test_sequence = "0030000300"  # Ejemplo de secuencia de prueba

# Calculamos el número de dígitos en la secuencia de prueba
N = len(test_sequence)

# La probabilidad de la secuencia completa es el producto de la probabilidad de cada dígito
prob_sequence = prob_digit ** N

# Calculamos la perplejidad basada en la probabilidad de la secuencia
def calculate_perplexity(N, prob_sequence):
    return (1 / prob_sequence) ** (1 / N)

# Cálculo de perplejidad
perplexity = calculate_perplexity(N, prob_sequence)

print(f"Perplejidad de la secuencia '{test_sequence}': {perplexity:.2f}")


Ahora, la perplejidad es 10, lo cual es consistente con un modelo donde todas las opciones (los dígitos del 0 al 9) son igualmente probables y la secuencia sigue una distribución uniforme. Esto refleja la idea de que hay 10 opciones posibles para cada dígito, lo que da lugar a una perplejidad de 10 para la secuencia.

Explicación de las correcciones:

1. **Probabilidad de la secuencia completa**: En lugar de usar la probabilidad de un solo dígito, calculamos la probabilidad de la secuencia completa como el producto de las probabilidades de cada dígito. Esto es $\left( \frac{1}{10} \right)^N$ para una secuencia de $N$ dígitos, ya que cada dígito tiene una probabilidad de $\frac{1}{10}$.
   
2. **Cálculo de la perplejidad**: Usamos la fórmula correcta para la perplejidad, que toma en cuenta la probabilidad de la secuencia completa y la normaliza por el número de dígitos  $N$.


La **perplejidad** como **weighted average branching factor** (factor de ramificación promedio ponderado) es una interpretación útil en el procesamiento del lenguaje natural (NLP). 
El **factor de ramificación** se refiere a la cantidad de posibles palabras que pueden seguir a una palabra dada en un lenguaje. 

La **perplejidad** refleja la cantidad promedio de opciones que tiene un modelo al predecir la siguiente palabra o símbolo en una secuencia.


1. **Situación 1: Distribución sesgada**

   Supongamos que el número **0** es mucho más común que otros números. Si el número **0** aparece el 90% de las veces, y los demás números tienen una probabilidad del 1% cada uno, el factor de ramificación efectivo será mucho menor porque el sistema puede predecir el **0** con más confianza.

   ```python
   import math

   # Probabilidad sesgada: 90% para '0', 1% para cada otro número
   prob_0 = 0.9
   prob_other = 0.01

   # Secuencia de prueba donde el '0' ocurre frecuentemente
   test_sequence = [0, 0, 0, 0, 3, 0, 0, 0, 0, 0]
   N = len(test_sequence)

   # Calculamos la probabilidad de la secuencia
   prob_sequence = 1
   for digit in test_sequence:
       if digit == 0:
           prob_sequence *= prob_0
       else:
           prob_sequence *= prob_other

   # Calculamos la perplejidad
   perplexity = (1 / prob_sequence) ** (1 / N)
   print(f"Perplejidad de la secuencia sesgada: {perplexity:.2f}")
   ```

   **Salida esperada**:
   ```
   Perplejidad de la secuencia sesgada: 1.81
   ```

**Ejemplo 2: Lenguaje artificial con palabras**

Supongamos que tenemos un lenguaje artificial con 3 posibles palabras: **A**, **B** y **C**, con las siguientes probabilidades:

- $P(A) = 0.7$
- $P(B) = 0.2$
- $P(C) = 0.1$

El factor de ramificación promedio ponderado nos dirá cuántas opciones tiene el modelo en promedio cuando predice la siguiente palabra.

**Paso 1: Probabilidad de la secuencia**

Para una secuencia de prueba $S = [A, A, B, C, A]$, calculamos la probabilidad de cada palabra según su distribución:

```python
import math

# Distribución de probabilidades
prob_A = 0.7
prob_B = 0.2
prob_C = 0.1

# Secuencia de prueba
test_sequence = ['A', 'A', 'B', 'C', 'A']
N = len(test_sequence)

# Calculamos la probabilidad de la secuencia
prob_sequence = 1
for word in test_sequence:
    if word == 'A':
        prob_sequence *= prob_A
    elif word == 'B':
        prob_sequence *= prob_B
    elif word == 'C':
        prob_sequence *= prob_C

# Calculamos la perplejidad
perplexity = (1 / prob_sequence) ** (1 / N)
print(f"Perplejidad de la secuencia de palabras: {perplexity:.2f}")
```

**Salida esperada**:
```
Perplejidad de la secuencia de palabras: 1.86
```

En este ejemplo, la perplejidad es baja porque el modelo está bastante seguro de predecir la palabra **A**, que ocurre con mayor frecuencia.



#### Ejercicios

**Ejercicio 1: Factor de ramificación en lenguaje determinista**

Imagina un lenguaje artificial en el que solo hay dos posibles palabras: **X** y **Y**. La palabra **X** ocurre con una probabilidad de 95%, y **Y** con 5%.

1. ¿Cuál sería la perplejidad de una secuencia que consiste únicamente en palabras **X**?
2. ¿Cuál sería la perplejidad de una secuencia con un solo **Y** seguido de muchas **X**?

Escribe un código para calcular la perplejidad para ambas secuencias.

**Ejercicio 2: Factor de ramificación en un corpus real**

Toma un corpus pequeño de texto (por ejemplo, una frase en inglés). Crea modelos de lenguaje de unigramas y bigramas utilizando probabilidades calculadas a partir de las frecuencias de las palabras en el corpus.

1. Calcula la perplejidad de una secuencia de prueba utilizando el modelo de unigramas.
2. Calcula la perplejidad utilizando un modelo de bigramas.
3. Compara las perplejidades. ¿Qué modelo predice mejor la secuencia?

```python
import nltk
from nltk.util import ngrams
from collections import Counter

# Frase ejemplo
corpus = "the cat sat on the mat the cat sat on the mat"
tokens = corpus.split()

# Unigramas
unigram_freq = Counter(tokens)
total_unigrams = sum(unigram_freq.values())

# Probabilidad de unigramas
unigram_probs = {word: freq / total_unigrams for word, freq in unigram_freq.items()}

# Bigramas
bigrams = list(ngrams(tokens, 2))
bigram_freq = Counter(bigrams)

# Probabilidad de bigramas
bigram_probs = {}
for bigram, count in bigram_freq.items():
    bigram_probs[bigram] = count / unigram_freq[bigram[0]]

# Secuencia de prueba
test_sequence = "the cat sat on the mat".split()

# Calcular perplejidad unigramas
def unigram_perplexity(test_sequence, unigram_probs):
    N = len(test_sequence)
    prob_sequence = 1
    for word in test_sequence:
        prob_sequence *= unigram_probs.get(word, 1e-10)  # evitar probabilidad 0
    return (1 / prob_sequence) ** (1 / N)

# Calcular perplejidad bigramas
def bigram_perplexity(test_sequence, bigram_probs):
    N = len(test_sequence) - 1
    prob_sequence = 1
    bigrams = list(ngrams(test_sequence, 2))
    for bigram in bigrams:
        prob_sequence *= bigram_probs.get(bigram, 1e-10)
    return (1 / prob_sequence) ** (1 / N)

# Perplejidad con unigramas
unigram_ppl = unigram_perplexity(test_sequence, unigram_probs)
print(f"Perplejidad del modelo de Unigramas: {unigram_ppl:.2f}")

# Perplejidad con bigramas
bigram_ppl = bigram_perplexity(test_sequence, bigram_probs)
print(f"Perplejidad del modelo de Bigramas: {bigram_ppl:.2f}")
```

**Ejercicio 3: Modelo de trigramas**

Extiende el ejemplo anterior para implementar un modelo de trigramas. Calcula la perplejidad de un modelo de trigramas y compáralo con los modelos de unigramas y bigramas.


In [ ]:
## Tus respuestas

### Muestreo de oraciones de un modelo de lenguaje

Una forma importante de visualizar el tipo de conocimiento que encarna un modelo de lenguaje es mediante el muestreo. Muestrear de una distribución significa elegir puntos aleatorios según su probabilidad. Por lo tanto, muestrear de un modelo de lenguaje que representa una distribución sobre oraciones significa generar algunas oraciones, eligiendo cada una según su probabilidad, tal como la define el modelo. Así, es más probable que generemos oraciones que el modelo considera de alta probabilidad y menos probable que generemos oraciones que el modelo considera de baja probabilidad.

Esta técnica de visualizar un modelo de lenguaje mediante muestreo fue sugerida por primera vez hace mucho tiempo por Shannon (1948) y Miller y Selfridge (1950). Es más sencillo visualizar cómo funciona esto en el caso de los unigramas. 

Imagina que todas las palabras del idioma inglés están distribuidas a lo largo de una línea que representa el espacio de probabilidad entre 0 y 1. Cada palabra ocupa un intervalo en esa línea, y el tamaño de cada intervalo es proporcional a la frecuencia con la que aparece la palabra en el idioma. Para generar una palabra aleatoria, elegimos un número aleatorio entre 0 y 1, encontramos dónde cae ese número en la línea de probabilidad, y seleccionamos la palabra cuyo intervalo contiene ese valor. Repetimos este proceso, eligiendo números aleatorios y generando palabras, hasta que generamos el token especial de final de oración `</s>`.

Este método puede extenderse para generar secuencias de palabras, como bigramas. Comenzamos generando aleatoriamente un bigrama que empiece con el token de inicio de oración `<s>`, de acuerdo con la probabilidad de los bigramas. Supongamos que la segunda palabra del bigrama es `w`. Luego, para generar la siguiente palabra, elegimos un bigrama que comience con `w` (nuevamente, basado en las probabilidades de los bigramas). Este proceso continúa, eligiendo el siguiente bigrama que comience con la palabra generada más recientemente, hasta que el proceso finalice con el token `</s>`.


**Ejemplo de generación de palabras aleatorias**

1. **Distribución de palabras en el intervalo [0, 1]:**
   Supongamos que tenemos un vocabulario muy simplificado con las siguientes palabras: `['the', 'dog', 'barks', 'in', 'the', 'park', '</s>']`. Aquí también incluimos el token de final de oración `</s>`. La probabilidad de cada palabra depende de su frecuencia en el idioma.

   Podemos asignar intervalos en la línea de probabilidad [0, 1] de la siguiente manera (usando probabilidades simplificadas para el ejemplo):

   - `the`: Probabilidad 0.4 --> Intervalo `[0.0, 0.4)`
   - `dog`: Probabilidad 0.2 --> Intervalo `[0.4, 0.6)`
   - `barks`: Probabilidad 0.1 --> Intervalo `[0.6, 0.7)`
   - `in`: Probabilidad 0.1 --> Intervalo `[0.7, 0.8)`
   - `park`: Probabilidad 0.05 --> Intervalo `[0.8, 0.85)`
   - `</s>`: Probabilidad 0.15 --> Intervalo `[0.85, 1.0)`

2. **Generación de una palabra:**
   Supongamos que generamos un número aleatorio entre 0 y 1, y el número es `0.32`. Este número cae en el intervalo de la palabra `the`, por lo que la primera palabra seleccionada es **the**.

3. **Continuar generando palabras:**
   Generamos otro número aleatorio, digamos `0.72`. Este número cae en el intervalo de la palabra `in`, por lo que la siguiente palabra seleccionada es **in**.

4. **Finalizar la oración:**
   Continuamos generando números aleatorios y seleccionando palabras hasta que finalmente generamos un número en el intervalo de `</s>`. Esto indica que hemos llegado al final de la oración.

**Extensión a bigramas**

Ahora, extendamos este concepto para trabajar con **bigramas**.

1. **Distribución de bigramas:**
   En lugar de asignar probabilidades a palabras individuales, asignamos probabilidades a secuencias de dos palabras (bigramas). Por ejemplo, supongamos que tenemos las siguientes probabilidades de bigramas basadas en un corpus de texto:

   - `<s> the`: 0.5 --> Intervalo `[0.0, 0.5)`
   - `<s> dog`: 0.3 --> Intervalo `[0.5, 0.8)`
   - `<s> park`: 0.2 --> Intervalo `[0.8, 1.0)`

   Así, la primera palabra de la oración es elegida según el bigrama que empieza con `<s>`. Generamos un número aleatorio, digamos `0.4`. Este número cae en el intervalo de `<s> the`, por lo que generamos el bigrama **the** como la primera palabra.

2. **Continuar con bigramas:**
   Ahora, queremos generar un bigrama que comience con la palabra `the`. Supongamos que las probabilidades para los bigramas que comienzan con `the` son:

   - `the dog`: 0.4 --> Intervalo `[0.0, 0.4)`
   - `the park`: 0.6 --> Intervalo `[0.4, 1.0)`

   Generamos otro número aleatorio, digamos `0.3`. Este número cae en el intervalo de `the dog`, por lo que el siguiente bigrama es **dog**. Ahora tenemos la secuencia `"<s> the dog"`.

3. **Continuar hasta finalizar:**
   Supongamos que las probabilidades de los bigramas que comienzan con `dog` son:

   - `dog barks`: 0.7 --> Intervalo `[0.0, 0.7)`
   - `dog </s>`: 0.3 --> Intervalo `[0.7, 1.0)`

   Generamos otro número aleatorio, digamos `0.9`, lo que nos lleva al bigrama `dog </s>`. Esto indica el final de la oración.


Usando este proceso, generamos la oración:

```
<s> the dog </s>
```

El proceso de selección aleatoria a lo largo de la línea de probabilidad para palabras y bigramas nos permite generar oraciones de manera coherente, basadas en las probabilidades observadas en un corpus de texto. Cada nueva palabra o secuencia de palabras (bigrama) es seleccionada en función de las frecuencias relativas de las secuencias precedentes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import Counter

# Función para calcular las probabilidades de unigramas a partir de un texto
def calculate_unigram_probs(text):
    words = text.split()
    total_words = len(words)
    word_counts = Counter(words)
    vocab = list(word_counts.keys())
    probs = [word_counts[word] / total_words for word in vocab]
    cumulative_probs = np.cumsum(probs)
    return vocab, probs, cumulative_probs

# Función para generar una palabra basada en un número aleatorio entre 0 y 1
def generate_word(vocab, cumulative_probs):
    rand_val = random.random()  # Elegimos un valor aleatorio entre 0 y 1
    for i, cum_prob in enumerate(cumulative_probs):
        if rand_val <= cum_prob:
            return vocab[i]

# Función para generar una oración completa
def generate_sentence(vocab, cumulative_probs):
    sentence = []
    while True:
        word = generate_word(vocab, cumulative_probs)
        if word == '</s>':  # Si obtenemos el token de fin de oración, terminamos
            break
        sentence.append(word)
    return ' '.join(sentence)

# Función para visualizar la distribución de unigramas
def visualize_unigram_sampling(vocab, probs, cumulative_probs):
    fig, ax = plt.subplots(figsize=(12, 3))  # Ajustamos el tamaño del gráfico
    
    # Graficamos las barras que representan los intervalos de probabilidad
    for i, word in enumerate(vocab):
        ax.barh(0, cumulative_probs[i], color='blue', alpha=0.5, edgecolor='black', 
                left=(cumulative_probs[i-1] if i > 0 else 0), height=0.5)
    
    # Mostrar el número de probabilidad acumulada en el eje X
    ax.set_xticks(np.round(cumulative_probs, 2))
    ax.set_xticklabels(np.round(cumulative_probs, 2), fontsize=10)
    ax.set_yticks([])
    
    # Etiquetar las palabras encima de las barras
    for i, word in enumerate(vocab):
        mid_point = (cumulative_probs[i-1] if i > 0 else 0) + (probs[i] / 2)
        plt.text(mid_point, 0.1, word, fontsize=12, va='center', ha='center', color='black')
    
    ax.set_xlim(0, 1)
    plt.title('Visualización de la distribución de muestreo de unigramas', fontsize=14)
    plt.show()

# Texto de ejemplo para generar el modelo de unigramas
text = "the of a to in the the of a in polyphonic however to in </s>"

# Calculamos las probabilidades de unigramas y las probabilidades acumuladas
vocab, probs, cumulative_probs = calculate_unigram_probs(text)

# Generar una oración de ejemplo
sentence = generate_sentence(vocab, cumulative_probs)
print("Oración generada:", sentence)

# Visualizar la distribución de probabilidades acumuladas
visualize_unigram_sampling(vocab, probs, cumulative_probs)


El codigo muestra una visualización utilizando un modelo de unigramas calculado a partir de un texto dado. 

#### Técnicas de muestreo

#### 1. **Muestreo aleatorio puro**
En este método, la próxima palabra se selecciona aleatoriamente de acuerdo con la distribución de probabilidad dada por el modelo. Esto es similar a lanzar un dado donde cada cara tiene un peso diferente según la probabilidad de cada palabra.

**Ejemplo**

Supongamos que el modelo ha calculado las siguientes probabilidades para las próximas palabras:
- `P(cat) = 0.05`
- `P(is) = 0.04`
- `P(jumping) = 0.03`
- `P(on) = 0.02`
- `P(the) = 0.01`

El modelo genera un número aleatorio entre 0 y 1. Supongamos que el número aleatorio generado es `0.045`. El modelo seleccionaría "is" porque su intervalo de probabilidad está entre `0.05 - 0.01 = 0.04` y `0.05`.

En este caso, "is" se selecciona. Esto introduce variabilidad en el texto, pero también puede resultar en oraciones que a veces no tienen sentido, como "The is jumping."

#### 2. **Muestreo top-k**

El muestreo top-k selecciona solo las `k` palabras más probables y descarta las demás, renormalizando la distribución de probabilidad entre las palabras seleccionadas.

**Ejemplo**
Supongamos que `k = 4`. Las palabras más probables con sus probabilidades son:
- "cat" (`P(cat) = 0.05`)
- "is" (`P(is) = 0.04`)
- "jumping" (`P(jumping) = 0.03`)
- "on" (`P(on) = 0.02`)

Las demás palabras se descartan. Luego, renormalizamos estas probabilidades entre estas cuatro palabras:
- `P'(cat) = 0.05 / (0.05 + 0.04 + 0.03 + 0.02) = 0.31`
- `P'(is) = 0.25`
- `P'(jumping) = 0.19`
- `P'(on) = 0.12`

El modelo selecciona una de estas palabras renormalizadas, digamos "cat", generando un texto más coherente, como "The cat is jumping." Sin embargo, esto puede reducir la diversidad del texto.

####  3. **Muestreo top-p**
En top-p muestreo, en lugar de un número fijo de palabras (`k`), seleccionamos un subconjunto de palabras cuya suma acumulada de probabilidades es al menos `p`.

**Ejemplo**

Supongamos que establecemos `p = 0.7`. Consideremos las probabilidades:
- "cat" (`P(cat) = 0.05`)
- "is" (`P(is) = 0.04`)
- "jumping" (`P(jumping) = 0.03`)
- "on" (`P(on) = 0.02`)
- "the" (`P(the) = 0.01`)
- "roof" (`P(roof) = 0.005`)
- ...

El modelo seleccionará palabras hasta que la suma acumulada alcance o supere 0.7. En este caso, se seleccionarán las palabras "cat," "is," "jumping," y "on" porque su suma (`0.05 + 0.04 + 0.03 + 0.02 = 0.14`) supera 0.7 cuando se consideran las siguientes palabras más probables.

Esto permite la inclusión de palabras como "roof" si encajan en el contexto, manteniendo el equilibrio entre diversidad y coherencia. Por ejemplo, puede generar la oración: "The cat is jumping on the roof."

Estas técnicas de muestreo permiten controlar la coherencia y diversidad en la generación de texto. Por ejemplo:
- **Muestreo aleatorio puro** puede dar lugar a textos inesperados como "The is jumping."
- **Top-k** podría generar "The cat is jumping," pero de manera predecible.
- **Top-p** puede generar textos como "The cat is jumping on the roof," agregando variabilidad sin sacrificar demasiada coherencia.

**Desafíos en el muestreo de oraciones**

1. **Equilibrio entre coherencia y diversidad**:
   - El modelo debe evitar la selección de palabras demasiado inesperadas, como "The is jumping on."
   - Con top-k o top-p, debe evitar la monotonía, permitiendo cierta creatividad como "The cat is sleeping on the roof."

2. **Probabilidades extremadamente bajas**:
   - El modelo puede evitar palabras con muy baja probabilidad, como "unicorn" en una oración sobre gatos y techos, a menos que sean necesarias para el contexto.

3. **Tamaño del vocabulario**:
   - A medida que crece el vocabulario, es fundamental manejar palabras raras, permitiendo combinaciones como "The cat is jumping."


### Ejercicios

**Ejercicio 1: Modificar el modelo de unigramas para manejar palabras desconocidas**

En este ejercicio, vas a agregar una funcionalidad para manejar palabras desconocidas (OOV). Usa un vocabulario cerrado que solo incluya palabras vistas en el conjunto de entrenamiento, y reemplaza las palabras desconocidas en el conjunto de prueba con un token especial `<UNK>`.

1. Modifica el código para generar una oración utilizando un vocabulario cerrado. Si una palabra desconocida aparece en el corpus de prueba, debe ser reemplazada por `<UNK>`.
2. Prueba con un corpus de prueba que contenga una palabra fuera del vocabulario, por ejemplo, `extraterrestrial`.

Pista: Puedes utilizar la función `convert_to_unk` del ejemplo anterior.


**Ejercicio 2: Implementar un modelo de bigrama**

En este ejercicio, vas a extender el código actual para trabajar con **bigramas**. Un modelo de bigramas elige la siguiente palabra en función de la palabra anterior.

1. Modifica el código para implementar un modelo de bigramas.
2. Usa el corpus de entrenamiento para calcular las probabilidades de los bigramas.
3. Genera una oración utilizando el modelo de bigramas, comenzando con un token especial `<s>`.
4. Visualiza la distribución de las probabilidades de los bigramas de manera similar a cómo se visualizan los unigramas.

##### Pistas:
- Usa `zip` para generar los bigramas del corpus.
- Asegúrate de incluir un token de inicio `<s>` en cada oración para iniciar el muestreo del bigrama.


**Ejercicio 3: Muestreo con suavizado**

En los modelos de n-gramas, cuando ciertas combinaciones de palabras no aparecen en el conjunto de entrenamiento, tienen una probabilidad de cero. En este ejercicio, vas a implementar **suavizado de Laplace** (add-one smoothing) para evitar probabilidades de cero.

1. Implementa el suavizado de Laplace en el modelo de unigramas.
2. Modifica el código para que todas las palabras, incluso las que no están en el corpus, tengan una pequeña probabilidad mayor a cero.
3. Genera una oración con el modelo suavizado.

##### Pista:
El suavizado de Laplace implica sumar 1 a la frecuencia de cada palabra y ajustar la suma total de palabras en consecuencia.


**Ejercicio 4: Visualización comparativa de unigramas y bigrama**

Este ejercicio consiste en comparar las distribuciones generadas por un modelo de unigramas y un modelo de bigramas.

1. Modifica el código para que genere dos gráficos: uno para la visualización de las probabilidades acumuladas de unigramas y otro para bigramas.
2. Compara cómo cambia la distribución entre los dos modelos.
3. Discute cómo los modelos de unigramas y bigramas afectan la coherencia de las oraciones generadas.

##### Pistas:
- Al generar bigramas, debes calcular las probabilidades condicionales de una palabra dado la anterior.
- Usa `Counter` para contar las ocurrencias de bigramas en el corpus.


**Ejercicio 5: Generalización y sparsity**

El **problema de sparsity** ocurre cuando un modelo de n-gramas no ha visto ciertos n-gramas en el conjunto de entrenamiento. En este ejercicio, vas a explorar este problema con un modelo de bigramas.

1. Entrena un modelo de bigramas en un corpus pequeño.
2. Intenta generar oraciones con el modelo. ¿Qué sucede cuando intentas generar una oración con una combinación de palabras que no aparece en el conjunto de entrenamiento?
3. Implementa un mecanismo que detecte estas combinaciones no vistas y las maneje adecuadamente (por ejemplo, generando un token `<UNK>` para secuencias no vistas).

**Ejercicio 6: Muestreo condicional basado en frecuencias**

1. Usa un modelo de bigramas para generar oraciones, pero esta vez ajusta la probabilidad de las palabras en función de la palabra anterior.
2. Usa una técnica similar a la de los unigramas, pero ahora al generar cada palabra, elige un bigrama en función de la última palabra generada.

##### Pistas:
- Calcula las probabilidades condicionales $P(w_n | w_{n-1})$.
- Usa el valor aleatorio para seleccionar el bigrama según las probabilidades acumuladas, como se hizo con los unigramas.


**Ejercicio 7: Experimentación con vocabularios cerrados y abiertos**

1. Usa un vocabulario cerrado para entrenar el modelo de unigramas.
2. Luego, expande el vocabulario para incluir más palabras y usa un vocabulario abierto. Compara cómo cambian las oraciones generadas y su coherencia entre ambos enfoques.
3. Discute las ventajas y desventajas de usar vocabularios cerrados frente a vocabularios abiertos.


**Ejercicio 8: Generación de texto con diferentes N-gramas**

1. Modifica el código para que pueda generar oraciones utilizando unigramas, bigramas y trigramas.
2. Compara la coherencia de las oraciones generadas por cada modelo. ¿Qué modelo genera oraciones más coherentes y por qué?
3. ¿Qué sucede cuando entrenas con un modelo de trigramas en un corpus pequeño? Explora el problema del **sparsity**.

##### Pista:
El modelo de trigramas debe calcular las probabilidades de la palabra actual dado las dos palabras anteriores.


**Ejercicio 9: Exploración de subpalabras y OOV**

1. Implementa una función que divida palabras en subpalabras o letras individuales cuando no se encuentren en el vocabulario (es decir, maneja palabras OOV utilizando tokenización de subpalabras).
2. Prueba el modelo generando oraciones en las que aparezcan palabras OOV y comprueba si la tokenización de subpalabras ayuda a generar texto más coherente.


**Ejercicio 10: Comparación de probabilidades acumuladas en unigramas**

1. Crea una función que permita seleccionar un subconjunto del vocabulario y mostrar las probabilidades acumuladas solo para ese subconjunto.
2. Genera oraciones utilizando solo palabras del subconjunto y visualiza cómo cambian las probabilidades acumuladas.

##### Pista:
Modifica la función `visualize_unigram_sampling` para que acepte un subconjunto de palabras y sus probabilidades.


In [ ]:
## Tus respuestas

### Generalización y ceros

El modelo de n-gramas, como muchos modelos estadísticos, depende del corpus de entrenamiento. Una implicación de esto es que las probabilidades a menudo codifican hechos específicos sobre un corpus de entrenamiento dado. Otra implicación es que los n-gramas hacen un mejor trabajo de modelado del corpus de entrenamiento a medida que aumentamos el valor de $N$.

¡Podemos usar el método de muestreo  para visualizar ambos hechos! 

Para dar una intuición sobre el poder creciente de los n-gramas de mayor orden, se muestra muestra oraciones aleatorias generadas a partir de modelos de unigramas, bigramas, trigramas y 4-gramas entrenados en las obras de Shakespeare.

```
1 grama:
- To him swallowed confess hear both. Which. Of save on trail for are ay device and
rote life have
- Hill he late speaks; or! a more to leg less first you enter

2 gramas

- Why dost stand forth thy canopy, forsooth; he is this palpable hit the King Henry. Live
king. Follow.
- What means, sir. I confess she? then all sorts, he is trim, captain.


3 gramas

- Fly, and will rid me these news of price. Therefore the sadness of parting, as they say,
’tis done.
- This shall forbid it should be branded, if renown made it empty.

4 gramas

- King Henry. What! I will go seek the traitor Gloucester. Exeunt some of the watch. A
great banquet serv’d in;
- It cannot be but so.
```

Ocho oraciones generadas aleatoriamente a partir de cuatro n-gramas calculados a partir de las obras de Shakespeare. Todos los caracteres se mapearon a minúsculas y los signos de puntuación se trataron como palabras. El resultado se corrigió manualmente para la capitalización a fin de mejorar la legibilidad. 


Cuanto más largo sea el contexto sobre el que entrenamos el modelo, más coherentes son las oraciones. En las oraciones de unigramas, no hay relación coherente entre palabras ni puntuación final de la oración. Las oraciones de bigramas tienen cierta coherencia local de palabra a palabra (especialmente si consideramos que los signos de puntuación cuentan como una palabra). Las oraciones de trigramas y 4-gramas comienzan a parecerse mucho a Shakespeare. 


Para hacernos una idea de la dependencia de una gramática con respecto a su conjunto de entrenamiento, echemos un vistazo a una gramática de n-gramas entrenada en un corpus completamente diferente: el periódico *Wall Street Journal* (WSJ). Shakespeare y el *Wall Street Journal* están en inglés, por lo que podríamos esperar cierto solapamiento entre nuestros n-gramas para los dos géneros. 

El texto siguiente  muestra oraciones generadas por gramáticas de unigramas, bigramas y trigramas entrenadas con 40 millones de palabras del WSJ.

```
1-grama
Months the my and issue of year foreign new exchange’s september
were recession exchange new endorsed a acquire to six executives

2-gramas
Last December through the way to preserve the Hudson corporation N.
B. E. C. Taylor would seem to complete the major central planners one
point five percent of U. S. E. has already old M. X. corporation of living
on information such as more frequently fishing to keep her

3- gramas
They also point to ninety nine point six billion dollars from two hundred
four oh six three percent of the rates of interest stores as Mexico and
Brazil on market conditions

```

Compara estos ejemplos con el pseudo-Shakespeare del texto. Aunque ambos modelan "oraciones en inglés", claramente no hay superposición en las oraciones generadas, y poca superposición incluso en frases pequeñas. Los modelos estadísticos probablemente sean bastante inútiles como predictores si los conjuntos de entrenamiento y de prueba son tan diferentes como Shakespeare y el WSJ.

¿Cómo debemos abordar este problema al construir modelos de n-gramas? Un paso es asegurarse de usar un corpus de entrenamiento que tenga un género similar a la tarea que estamos tratando de realizar. Para construir un modelo de lenguaje para traducir documentos legales necesitamos un corpus de entrenamiento de documentos legales. Para construir un modelo de lenguaje para un sistema de preguntas y respuestas, necesitamos un corpus de entrenamiento de preguntas.

Es igualmente importante obtener datos de entrenamiento en el dialecto o variedad apropiados, especialmente al procesar publicaciones en redes sociales o transcripciones de habla. Por ejemplo, algunos tweets usarán características del inglés afroamericano (AAE), el nombre para las muchas variaciones del lenguaje utilizadas en las comunidades afroamericanas (King, 2020). Dichas características incluyen palabras como *finna*, un verbo auxiliar que marca el futuro inmediato, que no ocurre en otras variedades, o grafías como *den* para *then*, en tweets como este: 

`Bored af den my phone finna die!!!`

mientras que los tweets de lenguas basadas en inglés como el pidgin nigeriano tienen un vocabulario y patrones de n-gramas marcadamente diferentes al inglés estadounidense:

`@username R u a wizard or wat gan sef: in d mornin - u tweet, afternoon - u tweet, nyt gan u dey tweet. beta get ur IT placement wiv twitter`.
 
Emparejar géneros y dialectos no es suficiente. Nuestros modelos aún pueden estar sujetos al problema del **sparsity**. Para cualquier n-grama que haya ocurrido un número suficiente de veces, podríamos tener una buena estimación de su probabilidad. Pero debido a que cualquier corpus es limitado, algunas secuencias de palabras perfectamente aceptables en inglés están destinadas a faltar en él. 

Es decir, tendremos muchos casos de "n-gramas de probabilidad cero" supuestos que realmente deberían tener alguna probabilidad distinta de cero. Considera las palabras que siguen al bigrama *denied the* en el corpus Treebank3 del WSJ, junto con sus conteos:

```
denied the allegations: 5
denied the speculation: 2
denied the rumors: 1
denied the report: 1
```

Pero supongamos que nuestro conjunto de prueba tiene frases como:

```
denied the offer
denied the loan
```

¡Nuestro modelo estimará incorrectamente que `P(offer|denied the)` es 0! Estos ceros que nunca ocurren en el conjunto de entrenamiento pero sí en el conjunto de prueba son un problema por dos razones. Primero, su presencia significa que estamos subestimando la probabilidad de todo tipo de palabras que podrían ocurrir, lo que afectará negativamente el rendimiento de cualquier aplicación que queramos ejecutar en estos datos.

Segundo, si la probabilidad de alguna palabra en el conjunto de prueba es $0$, la probabilidad total del conjunto de prueba es $0$. Por definición, la perplejidad se basa en la probabilidad inversa del conjunto de prueba. Por lo tanto, si algunas palabras tienen probabilidad cero, no podemos calcular la perplejidad en absoluto, ¡ya que no podemos dividir entre 0!

¿Qué hacemos con los ceros? Hay dos soluciones, dependiendo del tipo de cero. Para las palabras cuya probabilidad de n-grama es cero porque ocurren en un contexto nuevo en el conjunto de prueba, como en el ejemplo de *denied the offer* mencionado anteriormente, introduciremos los algoritmos llamados suavizado o descuento. Los algoritmos de suavizado quitan un poco de masa de probabilidad de algunos eventos más frecuentes y se la dan a estos eventos no vistos. 



#### Palabras desconocidas

¿Qué hacemos con palabras que nunca hemos visto antes? Tal vez la palabra *kapumota* simplemente no ocurrió en nuestro conjunto de entrenamiento, pero aparece en el conjunto de prueba. Por lo general, no permitimos esta situación estipulando que ya conocemos todas las palabras que pueden ocurrir. En un sistema de vocabulario cerrado, el conjunto de prueba solo puede contener palabras de este léxico conocido y no habrá palabras desconocidas. 

Esto es lo que hacemos para los modelos de lenguaje neuronales. Para estos modelos, usamos tokens de subpalabras en lugar de palabras. Con la tokenización de subpalabras (como el algoritmo BPE), cualquier palabra desconocida puede modelarse como una secuencia de subpalabras más pequeñas, si es necesario, mediante una secuencia de letras individuales, por lo que nunca tenemos palabras desconocidas.

Cuando nos encontramos con palabras desconocidas en un modelo de lenguaje, es decir, palabras que nunca hemos visto en el conjunto de entrenamiento pero que aparecen en el conjunto de prueba (como el ejemplo de la palabra *kapumota*), surge un problema. 

Estas palabras desconocidas no tienen una representación en el modelo de lenguaje entrenado, lo que puede causar dificultades en la predicción y generación de texto. A continuación, detallo cómo abordamos esta situación en diferentes tipos de sistemas de vocabulario y por qué las técnicas modernas han solucionado este problema.


#### Ejemplo concreto:

Imagina que el modelo encuentra la palabra *kapumota* en el conjunto de prueba y no la ha visto antes. Si el modelo usa tokenización basada en subpalabras, podría descomponer la palabra en:

- *kapu*, *mo*, *ta*

Estas subpalabras podrían ser combinaciones de fragmentos conocidos por el modelo, lo que le permitiría generar una predicción o asignar una probabilidad a la palabra sin haberla visto antes.

El uso de la tokenización de subpalabras en modelos de lenguaje neuronales permite manejar de manera efectiva palabras nuevas o desconocidas. Esto ha eliminado muchos de los problemas asociados con vocabularios cerrados o palabras OOV, y es una de las razones por las que los modelos modernos son mucho más robustos y flexibles que los enfoques anteriores basados en palabras completas. 

Con la tokenización de subpalabras, el modelo nunca se enfrenta a palabras totalmente desconocidas, ya que siempre puede descomponerlas y generar una representación procesable.

Sin embargo, si nuestro modelo de lenguaje está usando palabras en lugar de tokens, tenemos que lidiar con palabras desconocidas o palabras fuera del vocabulario (out of vocabulary)(OOV): palabras que no hemos visto antes. El porcentaje de palabras OOV que aparecen en el conjunto de prueba se llama **tasa de OOV**. 

Una forma de crear un sistema de vocabulario abierto es modelar las posibles palabras desconocidas en el conjunto de prueba agregando una pseudo-palabra llamada `<UNK>`. Nuevamente, la mayoría de los modelos de lenguaje modernos son de vocabulario cerrado y no usan un token `<UNK>`. Pero cuando sea necesario, podemos entrenar las probabilidades de <UNK> convirtiendo el problema de nuevo en uno de vocabulario cerrado eligiendo un vocabulario fijo por adelantado:

1. Elige un vocabulario (lista de palabras) que sea fijo por adelantado.
2. Convierte en el conjunto de entrenamiento cualquier palabra que no esté en este conjunto (cualquier palabra OOV) en el token de palabra desconocida `<UNK>` en un paso de normalización de texto.
3. Estima las probabilidades de `<UNK>` a partir de sus conteos como cualquier otra palabra regular en el conjunto de entrenamiento.

La elección exacta de `<UNK>` tiene un efecto en la perplejidad. Un modelo de lenguaje puede lograr una baja perplejidad eligiendo un vocabulario pequeño y asignando una alta probabilidad a la palabra desconocida. Por lo tanto, las perplejidades solo se pueden comparar entre modelos de lenguaje con `<UNK>` si tienen los mismos vocabularios exactos.

### Ejercicios

**Ejercicio 1: Modificar el modelo de unigramas para manejar palabras desconocidas**

En este ejercicio, vas a agregar una funcionalidad para manejar palabras desconocidas (OOV). Usa un vocabulario cerrado que solo incluya palabras vistas en el conjunto de entrenamiento, y reemplaza las palabras desconocidas en el conjunto de prueba con un token especial `<UNK>`.

1. Modifica el código para generar una oración utilizando un vocabulario cerrado. Si una palabra desconocida aparece en el corpus de prueba, debe ser reemplazada por `<UNK>`.
2. Prueba con un corpus de prueba que contenga una palabra fuera del vocabulario, por ejemplo, `extraterrestrial`.

**Pista:** Puedes utilizar la función `convert_to_unk` del ejemplo anterior.


**Ejercicio 2: Implementar un modelo de bigrama**

En este ejercicio, vas a extender el código actual para trabajar con **bigramas**. Un modelo de bigramas elige la siguiente palabra en función de la palabra anterior.

1. Modifica el código para implementar un modelo de bigramas.
2. Usa el corpus de entrenamiento para calcular las probabilidades de los bigramas.
3. Genera una oración utilizando el modelo de bigramas, comenzando con un token especial `<s>`.
4. Visualiza la distribución de las probabilidades de los bigramas de manera similar a cómo se visualizan los unigramas.

**Pistas:**
- Usa `zip` para generar los bigramas del corpus.
- Asegúrate de incluir un token de inicio `<s>` en cada oración para iniciar el muestreo del bigrama.


**Ejercicio 3: Muestreo con suavizado**

En los modelos de n-gramas, cuando ciertas combinaciones de palabras no aparecen en el conjunto de entrenamiento, tienen una probabilidad de cero. En este ejercicio, vas a implementar **suavizado de Laplace** (Add-One Smoothing) para evitar probabilidades de cero.

1. Implementa el suavizado de Laplace en el modelo de unigramas.
2. Modifica el código para que todas las palabras, incluso las que no están en el corpus, tengan una pequeña probabilidad mayor a cero.
3. Genera una oración con el modelo suavizado.

**Pista:** El suavizado de Laplace implica sumar 1 a la frecuencia de cada palabra y ajustar la suma total de palabras en consecuencia.


**Ejercicio 4: Visualización comparativa de unigramas y bigrama**

Este ejercicio consiste en comparar las distribuciones generadas por un modelo de unigramas y un modelo de bigramas.

1. Modifica el código para que genere dos gráficos: uno para la visualización de las probabilidades acumuladas de unigramas y otro para bigramas.
2. Compara cómo cambia la distribución entre los dos modelos.
3. Discute cómo los modelos de unigramas y bigramas afectan la coherencia de las oraciones generadas.

**Pista:**
- Al generar bigramas, debes calcular las probabilidades condicionales de una palabra dado la anterior.
- Usa `Counter` para contar las ocurrencias de bigramas en el corpus.


**Ejercicio 5: Generalización y sparsity**

El **problema de sparsity** ocurre cuando un modelo de n-gramas no ha visto ciertos n-gramas en el conjunto de entrenamiento. En este ejercicio, vas a explorar este problema con un modelo de bigramas.

1. Entrena un modelo de bigramas en un corpus pequeño.
2. Intenta generar oraciones con el modelo. ¿Qué sucede cuando intentas generar una oración con una combinación de palabras que no aparece en el conjunto de entrenamiento?
3. Implementa un mecanismo que detecte estas combinaciones no vistas y las maneje adecuadamente (por ejemplo, generando un token `<UNK>` para secuencias no vistas).



**Ejercicio 6: Muestreo condicional basado en frecuencias**

1. Usa un modelo de bigramas para generar oraciones, pero esta vez ajusta la probabilidad de las palabras en función de la palabra anterior.
2. Usa una técnica similar a la de los unigramas, pero ahora al generar cada palabra, elige un bigrama en función de la última palabra generada.

**Pistas:**

- Calcula las probabilidades condicionales $P(w_n | w_{n-1})$.
- Usa el valor aleatorio para seleccionar el bigrama según las probabilidades acumuladas, como se hizo con los unigramas.


**Ejercicio 7: Experimentación con vocabularios cerrados y abiertos**

1. Usa un vocabulario cerrado para entrenar el modelo de unigramas.
2. Luego, expande el vocabulario para incluir más palabras y usa un vocabulario abierto. Compara cómo cambian las oraciones generadas y su coherencia entre ambos enfoques.
3. Discute las ventajas y desventajas de usar vocabularios cerrados frente a vocabularios abiertos.


**Ejercicio 8: Generación de texto con diferentes n-gramas**

1. Modifica el código para que pueda generar oraciones utilizando unigramas, bigramas y trigramas.
2. Compara la coherencia de las oraciones generadas por cada modelo. ¿Qué modelo genera oraciones más coherentes y por qué?
3. ¿Qué sucede cuando entrenas con un modelo de trigramas en un corpus pequeño? Explora el problema del **sparsity**.

**Pista:** El modelo de trigramas debe calcular las probabilidades de la palabra actual dado las dos palabras anteriores.


**Ejercicio 9: Exploración de subpalabras y OOV**

1. Implementa una función que divida palabras en subpalabras o letras individuales cuando no se encuentren en el vocabulario (es decir, maneja palabras OOV utilizando tokenización de subpalabras).
2. Prueba el modelo generando oraciones en las que aparezcan palabras OOV y comprueba si la tokenización de subpalabras ayuda a generar texto más coherente.



**Ejercicio 10: Comparación de probabilidades acumuladas en unigramas**

1. Crea una función que permita seleccionar un subconjunto del vocabulario y mostrar las probabilidades acumuladas solo para ese subconjunto.
2. Genera oraciones utilizando solo palabras del subconjunto y visualiza cómo cambian las probabilidades acumuladas.

**Pista:** Modifica la función `visualize_unigram_sampling` para que acepte un subconjunto de palabras y sus probabilidades.


In [ ]:
## Respuestas